In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [36]:
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [29]:
loaders = [
    TextLoader("./data/ai-story.txt"),
    TextLoader("./data/appendix-keywords.txt"),
]
docs = [loader.load()[0] for loader in loaders]
docs

[Document(page_content='Scikit Learn\n\nScikit-learn은 Python 언어를 위한 또 다른 핵심 라이브러리로, 기계 학습의 다양한 알고리즘을 구현하기 위해 설계되었습니다. 이 라이브러리는 2007년 David Cournapeau에 의해 프로젝트가 시작되었으며, 그 후로 커뮤니티의 광범위한 기여를 받아 현재까지 발전해왔습니다. Scikit-learn은 분류, 회귀, 군집화, 차원 축소 등 다양한 기계 학습 작업을 지원하며, 사용하기 쉬운 API와 함께 제공되어 연구자와 개발자가 복잡한 데이터 과학 문제를 해결할 수 있도록 돕습니다.\n\n핵심 특징 중 하나는 다양한 기계 학습 모델을 일관된 인터페이스로 제공한다는 점입니다. 이는 사용자가 알고리즘 간에 쉽게 전환할 수 있게 하여, 최적의 모델을 찾는 과정을 단순화합니다. 또한, Scikit-learn은 사전 처리, 모델 선택, 평가 지표 등 기계 학습 파이프라인의 다른 단계에 필요한 도구들을 포함하고 있습니다. 이는 연구자와 개발자가 데이터를 더 효율적으로 처리하고, 모델의 성능을 정확히 평가할 수 있게 해줍니다.\n\nScikit-learn의 강점은 그의 범용성에 있습니다. 이는 다양한 산업과 연구 분야에서 사용될 수 있으며, 소규모 데이터셋부터 대규모 데이터셋까지 다양한 크기의 데이터를 처리할 수 있습니다. 또한, 오픈 소스 프로젝트로서, Scikit-learn은 지속적인 개선과 업데이트가 이루어지며, 사용자 커뮤니티로부터의 피드백과 기여를 받아 발전해 나갑니다.\n\n기계 학습 분야에 있어서, Scikit-learn은 특히 초보자에게 친화적인 학습 자원을 제공함으로써, 복잡한 이론과 알고리즘을 이해하는 데 중요한 역할을 합니다. 이러한 접근성과 범용성은 Scikit-learn을 기계 학습을 시작하는 이들에게 인기 있는 선택지로 만들었습니다.\n\nNLP\n\nNLP(자연어 처리)는 인간의 언어를 이해하고 해석하는 컴퓨터 알고리즘과 기술의 집합입니다. 이 분야는 컴퓨터 

# 작은 청크 생성
- 대용량 정보를 검색할 때 더 작은 청크로 나눠서 임베딩하는 것이 유용함
- 이를 통해 의미론적으로 최대한 근접하게 포착하면서, 가능한 많은 맥락을 하위 단계로 전달할 수 있다
- `ParentDocumentRetriever`가 이 작업을 수행한다

In [37]:
import uuid

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=HuggingFaceBgeEmbeddings(),
)
store = InMemoryByteStore()

id_key = "doc_id"
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4())] * len(docs)
doc_ids

['be9ed566-69ff-469f-8c86-7656ec2692a7',
 'be9ed566-69ff-469f-8c86-7656ec2692a7']

In [38]:
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000)
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [39]:
parent_docs = []

for i, doc in enumerate(docs):
    _id = doc_ids[i]
    parent_doc = parent_text_splitter.split_documents([doc])
    for _doc in parent_doc:
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)

In [40]:
child_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)

In [41]:
print(f"분할된 parent_docs의 개수: {len(parent_docs)}")
print(f"분할된 child_docs의 개수: {len(child_docs)}")

분할된 parent_docs의 개수: 4
분할된 child_docs의 개수: 54


In [42]:
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

retriever.docstore.mset(list(zip(doc_ids, docs)))

In [43]:
retriever.vectorstore.similarity_search("Word2Vec의 정의")

[Document(page_content='Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.', metadata={'doc_id': 'b270f99b-9b5e-4f0d-932d-408c12fcac58', 'source': './data/ai-story.txt'}),
 Document(page_content='Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.', metadata={'doc_id': 'be9ed566-69ff-469f-8c86-7656ec2692a7', 'source': './data/ai-story.txt'}),
 Document(page_content=', SARIMA는 계절적 패턴을 고려해야 하는 경우에 사용됩니다. SARIMAX는 외부 요인을 모델에 포함시켜야 할 때 가장 적합한 선택입니다.\n\nWord2Vec\n\nWord2Vec은 자연어 처리(NLP) 분야에서 널리 사용되는 획기적인 단어 임베딩 기법 중 하나입니다. 2013년 Google의 연구팀에 의해 개발되었으며, 단어를 벡터 공간에 매핑함으로써 컴퓨터가 단어의 의미를 수치적으로 이해할 수 있게 합니다. Word2Vec의 핵심 아이디어는 비슷한 맥락에서 사용되는 단어들

In [44]:
retriever.vectorstore.similarity_search_with_relevance_scores("Word2Vec의 정의", k=3, score_threshold=0.5)

[(Document(page_content='Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.', metadata={'doc_id': 'b270f99b-9b5e-4f0d-932d-408c12fcac58', 'source': './data/ai-story.txt'}),
  0.9045879327309017),
 (Document(page_content='Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.', metadata={'doc_id': 'be9ed566-69ff-469f-8c86-7656ec2692a7', 'source': './data/ai-story.txt'}),
  0.9045879327309017),
 (Document(page_content=', SARIMA는 계절적 패턴을 고려해야 하는 경우에 사용됩니다. SARIMAX는 외부 요인을 모델에 포함시켜야 할 때 가장 적합한 선택입니다.\n\nWord2Vec\n\nWord2Vec은 자연어 처리(NLP) 분야에서 널리 사용되는 획기적인 단어 임베딩 기법 중 하나입니다. 2013년 Google의 연구팀에 의해 개발되었으며, 단어를 벡터 공간에 매핑함으로써 컴퓨터가 단어의 의미를 수치적으로 이

In [45]:
relevant_doc = retriever.get_relevant_documents("Word2Vec의 정의?")
print(len(relevant_doc))
print(len(relevant_doc[0].page_content))

1
5733


In [48]:
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr

print(len(retriever.get_relevant_documents("Word2Vec의 정의")))
# len(retriever.get_relevant_documents("Word2Vec의 정의")[0].page_content)

0


## 요약본을 벡터DB에 같이 저장
- 종종 청크보다 더 정확하게 정보를 추출할 수 있다

In [49]:
import os
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=2048,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template(
        "Summarize the following document in Korean:\n\n{doc}"
    )
    | model
    | StrOutputParser()
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [50]:
summaries = chain.batch(docs, {"max_concurrency": 5})

In [51]:
summaries[0]

'Here is a summary of the document in Korean:\n\n**Scikit-Learn**\n\nScikit-Learn은 Python 언어를 위한 기계 학습 라이브러리입니다. 2007년에 시작된 프로젝트로, 다양한 알고리즘을 구현하고 있습니다. Scikit-Learn은 분류, 회귀, 군집화, 차원 축소 등 다양한 기계 학습 작업을 지원하며, 사용하기 쉬운 API를 제공합니다. 라이브러리의 강점은 다양한 알고리즘을 일관된 인터페이스로 제공하는 것입니다.\n\n**NLP**\n\nNLP는 자연어 처리의 분야입니다. 컴퓨터 알고리즘과 기술을 통해 인간의 언어를 이해하고 해석합니다. NLP의 주요 응용 분야로는 기계 번역, 감정 분석, 음성 인식, 자동 요약, 챗봇 개발 등이 있습니다. NLP의 핵심 과제는 자연어의 모호성과 다양성을 처리하는 것입니다.\n\n**SciPy**\n\nSciPy는 과학 계산을 위한 파이썬 라이브러리입니다. 2001년에 시작된 프로젝트로, 고급 수학 함수, 알고리즘, 그리고 편리한 도구를 제공합니다. SciPy는 NumPy 배열 객체 위에 구축되어 있으며, NumPy의 기능을 확장하여 다양한 과학 및 공학 분야에서 필요로 하는 특정한 기능과 유틸리티를 제공합니다.\n\n**Hugging Face**\n\nHugging Face는 자연어 처리(NLP) 분야에서 선도적인 역할을 하는 기술 회사입니다. 2016년에 설립된 이 회사는 특히 \'Transformers\' 라이브러리를 통해 큰 인기를 얻었습니다. Transformers 라이브러리는 BERT, GPT, RoBERTa, T5와 같은 최신의 변환기(Transformer) 기반 모델을 쉽게 사용할 수 있게 해주며, Python 프로그래밍 언어로 작성되어 있습니다.\n\n**Attention is all you need**\n\n"Attention Is All You Need"는 2017년에 발표된 논문으로, 변환기(Transformer) 모델의 아키텍처를 처음으로

In [52]:
summaries[1]

'Here is a summary of the document in Korean:\n\n**Semantic Search**\n\n* 의미론적 검색은 사용자의 질의를 분석하여 관련된 결과를 반환하는 검색 방식입니다.\n* 예시: "태양계 행성" 검색하면 "목성", "화성" 등과 같은 관련된 행성 정보를 반환합니다.\n\n**Embedding**\n\n* 임베딩은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다.\n* 예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n\n**Token**\n\n* 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다.\n* 예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n\n**Vectorizer**\n\n* 벡터라이저는 텍스트 데이터를 벡터 형식으로 변환하는 도구입니다.\n* 예시: "I love programming."라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n\n**VectorStore**\n\n* 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다.\n* 예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.\n\n**SQL**\n\n* SQL은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다.\n* 예시: SELECT * FROM users WHERE age > 18;은 18세 이상의 사용자 정보를 조회합니다.\n\n**CSV**\n\n* CSV는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다.\n* 예시: 이름, 나이, 직업이라는 헤더를 가진 CSV 파일에는 홍길동, 30, 개발자와 같은 데이터가 포함될 수 있습니다.\n\n**JSON**\n\n* JSON은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n* 예시: {"이름": "홍길동", "나이":

In [53]:
# 요약 정보를 저장할 벡터 저장소를 생성합니다.
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=HuggingFaceBgeEmbeddings())
# 부모 문서를 저장할 저장소를 생성합니다.
store = InMemoryByteStore()
# 문서 ID를 저장할 키 이름을 지정합니다.
id_key = "doc_id"
# 검색기를 초기화합니다. (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,  # 벡터 저장소
    byte_store=store,  # 바이트 저장소
    id_key=id_key,  # 문서 ID 키
)
# 문서 ID를 생성합니다.
doc_ids = [str(uuid.uuid4()) for _ in docs]

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [54]:
summary_docs = [
    # 요약된 내용을 페이지 콘텐츠로 하고, 문서 ID를 메타데이터로 포함하는 Document 객체를 생성합니다.
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(
        summaries
    )  # summaries 리스트의 각 요약과 인덱스에 대해 반복합니다.
]

In [55]:
retriever.vectorstore.add_documents(
    summary_docs
)  # 요약된 문서를 벡터 저장소에 추가합니다.

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [56]:
# RecursiveCharacterTextSplitter 객체를 생성합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

split_docs = []
split_docs_ids = []

for i, doc in enumerate(docs):
    _id = doc_ids[i]  # 현재 문서의 ID를 가져옵니다.
    # 현재 문서를 하위 문서로 분할합니다.
    split_doc = text_splitter.split_documents([doc])
    for _doc in split_doc:  # 분할된 문서에 대해 반복합니다.
        # 문서의 메타데이터에 ID를 저장합니다.
        _doc.metadata[id_key] = _id
        split_docs_ids.append(_id)
    split_docs.extend(split_doc)  # 분할된 문서를 리스트에 추가합니다.

In [57]:
retriever.vectorstore.add_documents(split_docs)

['749f750c-7cbf-4284-8c22-96ec88102201',
 '23c75217-2921-4653-852b-8a07a6c596ac',
 'dec45d63-97b3-4e7c-9196-c0505a50173d',
 '8ef458b7-3373-4379-9c9c-a61d8e5a5f8e',
 'be45a6ea-b564-4b92-8f0f-fec2a18e3140',
 '82a59b52-b83d-47a3-b1e7-8ddbb0c61fa4',
 '457440b0-30a7-4384-bace-0036b6a6fe57',
 'b2865e63-63c5-433b-9c83-f96a0e8e28d2',
 'e140b78a-042e-4d38-be2e-6115cf9ea54c',
 '8cced656-02f5-4527-8ccc-6bb642380092',
 '49b539fd-1b4c-43d2-89e3-9a899a17ac07',
 'ace96879-dc79-493e-b00f-b8b7903fce3e',
 '5374a86b-085f-4727-b6ca-9fda39b280c1',
 'daa05061-7ef7-4d10-b31a-f5967f1e58a6',
 '2eb6caf9-3100-4714-8023-a81df65617a7',
 '2a0dcdb1-134b-47e3-b76d-8c03eb9b97fa',
 '4e85311e-bef5-4c41-953d-b315adfe9381']

In [58]:
result_docs = vectorstore.similarity_search("Word2Vec의 정의가 뭐야?")
result_docs[0]

Document(page_content='Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.', metadata={'doc_id': 'd1baca23-2cb7-4cb9-91f1-d804ab0a866b', 'source': './data/ai-story.txt'})

In [59]:
retrieved_docs = retriever.get_relevant_documents("Word2Vec 의 정의가 뭐야?")
len(retrieved_docs)

1

In [60]:
len(retrieved_docs[0].page_content)

7482

# 가설 쿼리
- LLM은 문서에 대해 가정할 수 있는 질문 목록을 만들 수 있다
- 이를 이용하여 가설 질문을 임베딩하여 문서 내용을 더 깊이있게 탐색할 수 있다

In [61]:
functions = [
    {
        "name": "hypothetical_questions",  # 함수의 이름을 지정합니다.
        "description": "Generate hypothetical questions",  # 함수에 대한 설명을 작성합니다.
        "parameters": {  # 함수의 매개변수를 정의합니다.
            "type": "object",  # 매개변수의 타입을 객체로 지정합니다.
            "properties": {  # 객체의 속성을 정의합니다.
                "questions": {  # 'questions' 속성을 정의합니다.
                    "type": "array",  # 'questions'의 타입을 배열로 지정합니다.
                    "items": {
                        "type": "string"
                    },  # 배열의 요소 타입을 문자열로 지정합니다.
                },
            },
            "required": ["questions"],  # 필수 매개변수로 'questions'를 지정합니다.
        },
    }
]

In [62]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

output_parser = CommaSeparatedListOutputParser()

human_message = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="Generate a list of exactly 3 hypothetical questions that the below document could be used to answer.\n\n{doc}\n{format_instructions}",
        input_variables=["doc"],
        partial_variables={"format_instructions": output_parser.get_format_instructions()},
    )
)
prompt_chain = {"doc": lambda x: x.page_content} | ChatPromptTemplate.from_messages([human_message])

chain = (
    {"doc": lambda x: x.page_content}
    # 아래 문서를 사용하여 답변할 수 있는 가상의 질문을 정확히 3개 생성하도록 요청합니다. 이 숫자는 조정될 수 있습니다.
    | ChatPromptTemplate.from_messages([human_message])
    | model
    # 출력에서 "questions" 키에 해당하는 값을 추출합니다.
    | output_parser
)

In [63]:
prompt_chain.invoke(docs[0])
chain.invoke(docs[0])

['Here are three hypothetical questions that the provided document could be used to answer:\n\nScikit-Learn',
 'NLP',
 'SciPy',
 'HuggingFace',
 'Attention is all you need',
 'ARIMA',
 'SARIMA',
 'SARIMAX',
 "Word2Vec\n\nHere are the questions:\n\n1. What are the key features of Scikit-Learn and how does it differ from other machine learning libraries?\n2. How does HuggingFace's Transformers library contribute to the development of natural language processing (NLP) and what are its applications?\n3. What are the differences between ARIMA",
 'SARIMA',
 'and SARIMAX models in time series analysis',
 'and when would you use each one?']